In [ ]:
# only keep points in the focus states
focus_states_id_dict = {
    20: "jharkhand",
    22: "chhattisgarh",
    8: "rajasthan",
    23: "madhya pradesh",
    18: "assam",
    16: "tripura",
}

focus_states_filter = request_points_gdf["pc11_s_id"].isin(focus_states_id_dict.keys())
points_gdf_focus = request_points_gdf[focus_states_filter]
points_gdf_focus.shape

In [ ]:
torch_image = torch.from_numpy(image).float()
image = image.astype(np.float)
image = minmax_normalize_image(image)
torch_image = torch.from_numpy(image)
plt.imshow(image[3:0].transpose(1,2,0))
plt.savefig("TEST.png")
plt.close()

In [ ]:
# RASTERIO-only CustomDataSet. Used for speed experiments.

import rasterio
from rasterio.mask import mask

class CustomDataset(Dataset):
    def __init__(
        self,
        points,
        items,
        buffer,
        bands,
        resolution,
        dtype
    ):
        """
        Parameters
        ----------
        points : np.array
            Array of points to sample from
        items : list
            List of STAC items to sample from
        buffer : int
            Buffer in meters around each point to sample from
        bands : list
            List of bands to sample
        resolution : int
            Resolution of the image to sample

        Returns
        -------
        None
        """

        self.points = points
        self.items = items
        self.buffer = buffer
        self.bands = bands
        self.resolution = resolution

    def __len__(self):
        """Returns the number of points in the dataset"""

        return self.points.shape[0]

    def __getitem__(self, idx):
        """
        Parameters
        ----------
        idx : int
            Index of the point to get imagery for

        Returns
        -------
        out_image : torch.Tensor
            Image tensor of shape (C, H, W)
        """

        lon, lat = self.points[idx]
        stac_item = self.items[idx]

        if stac_item is None:
            print(f"Skipping {idx}: No STAC item given.")
            return None
        else:
            # get STAC item URL
            stac_item = stac_item.to_dict()

            # url = stac_item["assets"]["visual"]["href"]
            url = stac_item["assets"]["B02"]["href"]

            with rasterio.Env():
                with rasterio.open(url, "r") as f:
                    
                    crs = f.crs.to_string()
                    proj_latlon_to_stac = pyproj.Transformer.from_crs(4326, crs, always_xy=True)
                    lon, lat = proj_latlon_to_stac.transform(lon, lat)
                    point_geom = shapely.geometry.Point(lon, lat)

                    mask_geom = point_geom.buffer(self.buffer).envelope
                    mask_geom_dict = shapely.geometry.mapping(mask_geom)

                    try:
                        image, out_transform = mask(
                            f, [mask_geom_dict], crop=True, 
                        )
                    except ValueError as e:
                        if "Input shapes do not overlap raster." in str(e):
                            return None

            try:
                # image = image.astype(np.float)
                image = image / 255.0
                torch_image = torch.from_numpy(image).float()
                return torch_image

            except Exception as e:
                print(f"Skipping {idx}:", e)
                return None